In [ ]:
# For heavy-duty analysis

In [ ]:
# Array shape conventions:
# d = vector dimension (6, usually)
# T = number of time points (~10000, usually)
# N = number of trajectories simulated
# 1. Vector time series: (d, T)
# 2. Ensemble: (d, T, N)

In [ ]:
module_path = "../"
if findfirst(x -> x == module_path, LOAD_PATH) == nothing
    push!(LOAD_PATH, module_path)
end
using Revise

In [ ]:
using Utils
using Solve
using Analysis
using DSP
using Plots

In [ ]:
# some plotting parameters
tickfontsize = 11
labelfontsize = 14
legendfontsize = 10

In [ ]:
#=
Pseudocode for aggregate_data
    a dataset consists of metadata, a solution with no noise, and solution(s) with noise.

result = a dictionary ( key -> value )

for each unique key
    result[key] = nothing
    for each dataset that shares that key
        metadata, no_noise_sol, sol = dataset
        if not selector(metadata)
            continue
        end
        result[key] = aggregator(result[key], metadata, no_noise_sol, sol)
    end
    result[key] = postprocess(key, result[key])
end

return result
=#

In [ ]:
save_dir = "data/results"

function polarization_aggregator(r, metadata, no_noise_sol, sol)
    t, pol = extract_noise(sol; noisetype="npolarization", nsmooth=10, runs=:)
    return t, pol
end

function polarization_postprocess(key, r)
    # No post-processing.
    # You can also just leave this argument out entirely - this is what it does by default.
    return r
end

res = aggregate_data("data/results";
        selector=m->(m["lowercutoff"] < 1000), # Pick only datasets with cutoff < 1000
        key_func=(m,n,s)->m["lowercutoff"],
        aggregator=polarization_aggregator,
        postprocess=polarization_postprocess)

In [ ]:
plot()
for k=keys(res)
    t, pol = res[k]
    plot!(t, pol, label=string(k, "Hz"))
end
plot!(ylabel=string("Polarization"), xlabel="Time [s]")
plot!(ytickfontsize=tickfontsize, xtickfontsize=tickfontsize)
plot!(yguidefontsize=labelfontsize, xguidefontsize=labelfontsize)
plot!(legendfontsize=legendfontsize)

In [ ]:
metadata = get_random_metadata("data/results")
cutoffs = [key for key=keys(res)]
errors = [res[key][2] for key=keys(res)]
scatter(cutoffs, 1 .- [res[key][2][end] for key=cutoffs], yscale=:log10, label="Simulation")
cc = range(0, 3000, length=500)
nn = [1 .- predict_polarization(1., neutrongyro; Bnoise=metadata["Bnoise"], noiserate=metadata["noiserate"], lowercutoff=f) for f=cc]
plot!(cc, nn, label="Theory")
yaxis!("1/T_2 [s^-1]")
xaxis!("Highpass Filter Cutoff [Hz]")
plot!(ytickfontsize=tickfontsize, xtickfontsize=tickfontsize)
plot!(yguidefontsize=labelfontsize, xguidefontsize=labelfontsize)
plot!(legendfontsize=legendfontsize)

In [ ]:
data_directories = [joinpath(save_dir, s) for s=readdir(save_dir)]

In [ ]:
using Visualizations

In [ ]:
no_noise_sol, sol, metadata = load_data(data_directories[1]);

In [ ]:
xaxes, yaxes, scale = determine_axes(sol);

In [ ]:
anim = @animate for i = 1:3000
    frame = sol.u[1:3,i,:]
    blochplot(frame, xaxes[i], yaxes[i], scale * 3; title=string("t=", sol.t[i], "s"))
    xv, yv = get_quivers(frame, xaxes[i], yaxes[i], scale)
    plot!([0], [0], quiver=([xv[1]], [yv[1]]), label="East", subplot=1, seriescolor=:orange, seriestype=:quiver);
    plot!([0], [0], quiver=([xv[2]], [yv[2]]), label="North", subplot=1, seriescolor=:green, seriestype=:quiver);
    #x0, y0 = project(no_noise_sol.u[1:3,i,:], xaxes[i], yaxes[i])
    #scatter!([mean(x0)], [mean(y0)], markercolor=:red)
    end every 20
gif(anim, "anim_fps15.gif", fps = 15)